[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joycemsm/fashion-store-analysis/blob/main/sales_analysis_queries_SQL.ipynb)


In [11]:
!pip install pandasql
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=3f00edd22940e3eceadadaa6d2619469b1fc110792fdbca5c9b11aa67c589fd7
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [7]:
products = pd.read_csv('/content/dataset_fashion_store_products.csv')
sales = pd.read_csv('/content/dataset_fashion_store_sales.csv')
sales_items = pd.read_csv('/content/dataset_fashion_store_salesitems.csv')
customers = pd.read_csv('/content/dataset_fashion_store_customers.csv')
stock = pd.read_csv('/content/dataset_fashion_store_stock.csv')
campaigns = pd.read_csv('/content/dataset_fashion_store_campaigns.csv')
channels = pd.read_csv('/content/dataset_fashion_store_channels.csv')

# **Fashion Store - Sales Analysis (SQL)**

This notebook presents business-driven SQL queries focused on the Sales Analysis of the Fashion Store database.
It answers key performance questions using structured SQL logic, suitable for both technical and business audiences.

## **Summary**

1. **Which products have low sales volume?**
2. **Which products are below average in sales?**
3. **Are there underperforming product categories?**
4. **Did any product show a sales drop recently?**
5. **Which products are in stock but haven’t sold recently?**


## **How has sales performance evolved over time?**

Goal:
Track monthly sales performance using key metrics such as total sales, total revenue, and average ticket size.

Explanation:

*   Extracts the month from each sale using substr(sale_date, 1, 7).

*   Aggregates metrics:  

    * COUNT(sale_id) to measure the number of sales.

    * SUM(total_amount) for total revenue.

    * AVG(total_amount) for average ticket size.

*   Adds thousand separators to monetary values to improve data presentation.

In [20]:
query = """
SELECT
    substr(s.sale_date, 1, 7) AS month,
    COUNT(s.sale_id) AS total_sales,
    SUM(s.total_amount) AS total_revenue,
    ROUND(AVG(s.total_amount), 2) AS avg_ticket
FROM
    sales s
GROUP BY
    month
ORDER BY
    month;
"""
result = pysqldf(query)

# Converter para data + formatar mês
result['month'] = pd.to_datetime(result['month'], format='%Y-%m').dt.strftime('%B %Y')

# Formatar total_revenue e avg_ticket com separador de milhar
result['total_revenue'] = result['total_revenue'].map('{:,.2f}'.format)
result['avg_ticket'] = result['avg_ticket'].map('{:,.2f}'.format)

result

,month,total_sales,total_revenue,avg_ticket
0,April 2025,364,"133,392.44",366.46
1,May 2025,403,"141,922.21",352.16
2,June 2025,138,"48,922.01",354.51


In [21]:
result.to_csv('monthly_sales_summary.csv', index=False)

from google.colab import files
files.download('monthly_sales_summary.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Who are the customers at risk of churn?**

Goal:
Identify customers who are likely to churn due to inactivity, specifically those who haven’t made a purchase in the last 90 days.

Explanation:

*   Uses MAX(sale_date) to find the most recent purchase date per customer.

*   Calculates days since last purchase using JULIANDAY() function.

*   Filters customers with > 90 days since last purchase.

*   Orders customers by total spending in descending order.

*   Uses LEFT JOIN to include customers who may have never purchased.

In [25]:
# Query: Customers who haven't purchased in over 90 days
query = """
SELECT
    c.customer_id,
    c.age_range,
    c.country,
    MAX(s.sale_date) AS last_purchase_date,
    ROUND(JULIANDAY('now') - JULIANDAY(MAX(s.sale_date))) AS days_since_last_purchase
FROM
    customers c
LEFT JOIN
    sales s ON c.customer_id = s.customer_id
GROUP BY
    c.customer_id, c.age_range, c.country
HAVING
    days_since_last_purchase > 90
ORDER BY
    days_since_last_purchase DESC;
"""

result = pysqldf(query)
result

,customer_id,age_range,country,last_purchase_date,days_since_last_purchase
0,128,56-65,Netherlands,2025-04-04,97.0
1,172,26-35,Italy,2025-04-04,97.0
2,374,16-25,Italy,2025-04-04,97.0
3,545,16-25,Germany,2025-04-04,97.0
4,557,56-65,Spain,2025-04-04,97.0
5,655,56-65,Italy,2025-04-04,97.0
6,162,36-45,Germany,2025-04-05,96.0
7,202,36-45,Netherlands,2025-04-05,96.0
8,697,56-65,Germany,2025-04-05,96.0
9,706,36-45,France,2025-04-05,96.0


In [26]:
# Export to CSV
result.to_csv('customers_at_churn_risk.csv', index=False)

# Download file
from google.colab import files
files.download('customers_at_churn_risk.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Where are the most valuable customers?**

Goal:
Find the customers with the highest lifetime value (LTV), based on their total spend.

Explanation:

*   Uses SUM(total_amount) to calculate total spend per customer.

*   Adds COUNT(sale_id) to show total number of orders.

*   Calculates AVG(total_amount) as average ticket size.

*   Orders customers by total spending in descending order.

*   Includes customer profile details like age_range and country.

In [27]:
# Query: Identify top 20 customers by total spending (LTV)
query = """
SELECT
    c.customer_id,
    c.age_range,
    c.country,
    COUNT(s.sale_id) AS total_orders,
    SUM(s.total_amount) AS total_spent,
    ROUND(AVG(s.total_amount), 2) AS avg_ticket
FROM
    customers c
JOIN
    sales s ON c.customer_id = s.customer_id
GROUP BY
    c.customer_id, c.age_range, c.country
ORDER BY
    total_spent DESC
LIMIT 20;
"""

result = pysqldf(query)
result

,customer_id,age_range,country,total_orders,total_spent,avg_ticket
0,99,36-45,Spain,7,2678.59,382.66
1,872,26-35,Netherlands,5,2221.05,444.21
2,220,26-35,Germany,4,2180.51,545.13
3,816,16-25,Netherlands,4,1921.08,480.27
4,253,26-35,France,4,1882.24,470.56
5,277,16-25,Italy,4,1825.27,456.32
6,776,56-65,Portugal,4,1822.66,455.67
7,190,26-35,Italy,5,1722.06,344.41
8,564,26-35,Italy,3,1702.70,567.57
9,453,16-25,France,3,1665.51,555.17


In [28]:
# Export to CSV
result.to_csv('top_valuable_customers.csv', index=False)

# Download CSV
from google.colab import files
files.download('top_valuable_customers.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Which products have low sales volume?**

Goal:
Identify products that have sold the least in total units.  

Explanation:

*   Uses SUM(quantity) to calculate total units sold.

*   Orders results in ascending order to find low-performing products.

*   Brings product names using JOIN with the products table.

In [ ]:
query = """
SELECT
    products.product_id,
    products.product_name,
    SUM(sales_items.quantity) AS total_sold
FROM
    sales_items
JOIN
    products ON sales_items.product_id = products.product_id
GROUP BY
    products.product_id, products.product_name
ORDER BY
    total_sold ASC
LIMIT 10;
"""
result = pysqldf(query)
result

,product_id,product_name,total_sold
0,26,Soft Crew Shoes,1
1,64,Soft Sleeveless Set,1
2,67,Modern Crew Tee,1
3,87,Classic Silk Dress,1
4,377,Tailored High-Waist Shoes,1
5,379,Essential Cotton Trousers,1
6,424,Elegant Sleeveless Dress,1
7,58,Modern Sleeveless Dress,2
8,130,Classic Ribbed Set,2
9,218,Polished Satin Tee,2


In [15]:
result.to_csv('products_low_sales_volume.csv', index=False)

from google.colab import files
files.download('products_low_sales_volume.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Which products are below average in sales?**

Goal:
Identify products whose total sales volume is lower than the global product average.

Explanation:

*   Uses a CTE to compute total units sold per product.

*   Filters products below the average sales volume.

*   Useful to identify broadly underperforming products.

In [12]:
query = """
WITH sales_product AS (
	SELECT
	products.product_id,
    products.product_name,
    SUM(sales_items.quantity) AS total_quantity_sold
FROM sales_items
JOIN products ON sales_items.product_id = products.product_id
GROUP BY products.product_id, products.product_name
)
SELECT *
FROM sales_product
WHERE total_quantity_sold < (
	SELECT AVG(total_quantity_sold)
    FROM sales_product
)
ORDER BY total_quantity_sold ASC;
"""
result = pysqldf(query)
result

,product_id,product_name,total_quantity_sold
0,26,Soft Crew Shoes,1
1,64,Soft Sleeveless Set,1
2,67,Modern Crew Tee,1
3,87,Classic Silk Dress,1
4,377,Tailored High-Waist Shoes,1
...,...,...,...
263,430,Bold Linen Set,13
264,442,Bold Ribbed Dress,13
265,463,Bold Boxy Dress,13
266,472,Tailored Satin Trousers,13


In [14]:
# Exportar para CSV
result.to_csv('products_below_average_sales.csv', index=False)

# Baixar no Colab
from google.colab import files
files.download('products_below_average_sales.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Are there underperforming product categories?**

Goal:
Highlight categories with total sales below average.

Explanation:

*   Aggregates sales by category.

*   Filters categories with sales below the average of all categories.

*   Helps marketing prioritize attention.

In [ ]:
query = """
WITH sales_category AS (
    SELECT
        products.category,
        SUM(sales_items.quantity) AS total_sales
    FROM
        sales_items
    JOIN
        products ON sales_items.product_id = products.product_id
    GROUP BY
        products.category
)
SELECT *
FROM sales_category
WHERE total_sales < (
    SELECT AVG(total_sales) FROM sales_category
)
ORDER BY total_sales ASC;
"""
result = pysqldf(query)
result

,category,total_sales
0,Pants,1063


## **Did any product show a sales drop recently?**

Goal:
Compare month-over-month sales to detect declines.

Explanation:

*   Uses a CTE to calculate monthly sales.

*   LAG() compares current to previous month.

*   Filters for products where sales dropped month-to-month.

In [ ]:
query = """
WITH monthly_sales AS (
    SELECT
        p.product_id,
        p.product_name,
        substr(s.sale_date, 1, 7) AS month,
        SUM(si.quantity) AS total_sold
    FROM sales_items si
    JOIN sales s ON si.sale_id = s.sale_id
    JOIN products p ON si.product_id = p.product_id
    GROUP BY p.product_id, p.product_name, month
),
sales_with_lag AS (
    SELECT
        product_id,
        product_name,
        month,
        total_sold,
        LAG(total_sold) OVER (PARTITION BY product_id ORDER BY month) AS previous_month_sold
    FROM monthly_sales
)
SELECT *
FROM sales_with_lag
WHERE previous_month_sold IS NOT NULL
  AND total_sold < previous_month_sold
ORDER BY product_name, month;

"""
result = pysqldf(query)
result

,product_id,product_name,month,total_sold,previous_month_sold
0,490,Bold Boxy Shoes,2025-05,12,14
1,445,Bold Cotton Set,2025-05,10,20
2,474,Bold Cotton Shoes,2025-05,7,9
3,452,Bold Crew Dress,2025-05,8,9
4,454,Bold Crew Set,2025-05,4,12
...,...,...,...,...,...
328,158,Vintage Sleeveless Dress,2025-05,2,3
329,201,Vintage Sleeveless Tee,2025-05,7,12
330,201,Vintage Sleeveless Tee,2025-06,3,7
331,215,Vintage Sleeveless Trousers,2025-06,1,7


In [16]:
result.to_csv('products_with_recent_sales_drop.csv', index=False)

from google.colab import files
files.download('products_with_recent_sales_drop.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Which products are in stock but haven’t sold recently?**

Goal:
Identify products with stock available but no sales in the last 90 days.

Explanation:

*   Filters stock for products with positive quantity.

*   Excludes those sold in the last 90 days using a subquery.

*   Useful to detect potentially dead inventory.

In [ ]:
query = """
SELECT
    p.product_id,
    p.product_name,
    s.stock_quantity,
    s.country
FROM
    stock AS s
JOIN
    products AS p ON s.product_id = p.product_id
WHERE
    s.stock_quantity > 0
    AND p.product_id NOT IN (
        SELECT DISTINCT si.product_id
        FROM sales_items AS si
        JOIN sales AS sa ON si.sale_id = sa.sale_id
        WHERE sa.sale_date >= date('now', '-90 day')
    );
"""
result = pysqldf(query)
result

,product_id,product_name,stock_quantity,country
0,465,Bold Boxy Set,42,France
1,465,Bold Boxy Set,1,Germany


In [17]:
result.to_csv('products_in_stock_no_recent_sales.csv', index=False)

from google.colab import files
files.download('products_in_stock_no_recent_sales.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Conclusion**

This sales analysis provided actionable insights into the store’s product performance by answering business-critical questions through structured SQL.